## Preparation Step

### Data Preprocessing

Check library installation

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 29.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

ag_news_dataset = load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
train_dataset = ag_news_dataset["train"]
test_dataset = ag_news_dataset["test"]

Import necessary library

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00


In [5]:
import torch
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
print("PyTorch version:", torch.__version__)

PyTorch version: 2.0.0+cu118


Load dataset and tokenize it

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
def preprocess_data(dataset):
  texts = dataset['text']
  labels = dataset['label']
  
  # Tokenize
  # Note: length can be monified
  inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=64, return_tensors='pt')
  
  # Convert labels to a tensor
  labels = torch.tensor(labels)
  
  # Combine inputs and labels
  processed_dataset = list(zip(inputs['input_ids'], inputs['attention_mask'], inputs['token_type_ids'], labels))
  
  return processed_dataset


In [9]:
train_dataset = preprocess_data(train_dataset)
test_dataset = preprocess_data(test_dataset)

In [10]:
def collate_fn(batch):
  input_ids, attention_mask, token_type_ids, labels = zip(*batch)
  input_ids = torch.stack(input_ids)
  attention_mask = torch.stack(attention_mask)
  token_type_ids = torch.stack(token_type_ids)
  labels = torch.tensor(labels, dtype=torch.long)
  return input_ids, attention_mask, token_type_ids, labels


Create Dataloader for train, validation, and testing:

In [11]:
batch_size = 128

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


### Model Building

#### Import library:

In [12]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertModel

#### Teacher

VDCNN with pretrained BERT embeddings

In [13]:
class BasicConvResBlock(nn.Module):

  def __init__(self, input_dim=128, n_filters=256, kernel_size=3, padding=1, stride=1, shortcut=False, downsample=None):
    super(BasicConvResBlock, self).__init__()

    self.downsample = downsample
    self.shortcut = shortcut

    self.conv1 = nn.Conv1d(input_dim, n_filters, kernel_size=kernel_size, padding=padding, stride=stride)
    self.bn1 = nn.BatchNorm1d(n_filters)
    self.relu = nn.ReLU()
    self.conv2 = nn.Conv1d(n_filters, n_filters, kernel_size=kernel_size, padding=padding, stride=stride)
    self.bn2 = nn.BatchNorm1d(n_filters)

  def forward(self, x):

    residual = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)

    if self.shortcut:
      if self.downsample is not None:
        residual = self.downsample(x)
      out += residual

    out = self.relu(out)

    return out


In [14]:
class VDCNN(nn.Module):
  def __init__(self, num_classes, embedding_dim=768, depth=9):
    super(VDCNN, self).__init__()
    #self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.conv1 = nn.Conv1d(embedding_dim, 64, kernel_size=3, padding=1)
    self.layers = self._make_layers(depth)
    self.pool = nn.AdaptiveMaxPool1d(8)
    fc_layers = []
    # fully connected
    fc_layers.extend([nn.Linear(8 * 512, 2048), nn.ReLU()])
    fc_layers.extend([nn.Linear(2048, 2048), nn.ReLU()])
    fc_layers.extend([nn.Linear(2048, n_classes)])
    self.fc_layers = nn.Sequential(*fc_layers)

  def _make_layers(self, depth):
    layers = []

    if depth == 9:
      n_conv_block_64, n_conv_block_128, n_conv_block_256, n_conv_block_512 = 1, 1, 1, 1
    elif depth == 17:
      n_conv_block_64, n_conv_block_128, n_conv_block_256, n_conv_block_512 = 2, 2, 2, 2
    elif depth == 29:
      n_conv_block_64, n_conv_block_128, n_conv_block_256, n_conv_block_512 = 5, 5, 2, 2
    elif depth == 49:
      n_conv_block_64, n_conv_block_128, n_conv_block_256, n_conv_block_512 = 8, 8, 5, 3

    layers.append(BasicConvResBlock(input_dim=64, n_filters=64, kernel_size=3, padding=1))
    for _ in range(n_conv_block_64-1):
      layers.append(BasicConvResBlock(input_dim=64, n_filters=64, kernel_size=3, padding=1))
    layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1)) # l = initial length / 2

    ds = nn.Sequential(nn.Conv1d(64, 128, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(128))
    layers.append(BasicConvResBlock(input_dim=64, n_filters=128, kernel_size=3, padding=1, downsample=ds))
    for _ in range(n_conv_block_128-1):
      layers.append(BasicConvResBlock(input_dim=128, n_filters=128, kernel_size=3, padding=1))
    layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1)) # l = initial length / 4

    ds = nn.Sequential(nn.Conv1d(128, 256, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(256))
    layers.append(BasicConvResBlock(input_dim=128, n_filters=256, kernel_size=3, padding=1, downsample=ds))
    for _ in range(n_conv_block_256 - 1):
      layers.append(BasicConvResBlock(input_dim=256, n_filters=256, kernel_size=3, padding=1))
    layers.append(nn.MaxPool1d(kernel_size=3, stride=2, padding=1))

    ds = nn.Sequential(nn.Conv1d(256, 512, kernel_size=1, stride=1, bias=False), nn.BatchNorm1d(512))
    layers.append(BasicConvResBlock(input_dim=256, n_filters=512, kernel_size=3, padding=1, downsample=ds))
    for _ in range(n_conv_block_512 - 1):
      layers.append(BasicConvResBlock(input_dim=512, n_filters=512, kernel_size=3, padding=1))

    return nn.Sequential(*layers)

  def forward(self, x):
    #x = self.embedding(x)
    #x = x.transpose(1, 2)
    x = self.conv1(x)
    x = self.layers(x)
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layers(x)
    return x


In [15]:
class BERT_VDCNN(nn.Module):
  def __init__(self, n_classes, freeze_bert=True, depth=9):
    super(BERT_VDCNN, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.vdcnn = VDCNN(n_classes, depth=depth)
    if freeze_bert:
      for param in self.bert.parameters():
        param.requires_grad = False
  def forward(self, input_ids, attention_mask, token_type_ids):
    # load bert and feed in data
    outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    embeddings = outputs.last_hidden_state.permute(0, 2, 1)  # Swap the last two dimensions to match VDCNN input shape (batch_size, embed_size, seq_length)

    # bert as embeddings of the vdcnn
    logits = self.vdcnn(embeddings)

    return logits


#### Student

In [ ]:
# simple CNN model that will be trained and coached
class SimpleCNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, n_classes):
    super(SimpleCNN, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.conv1 = nn.Conv1d(embed_dim, 128, 3, padding=1)
    self.conv2 = nn.Conv1d(128, 256, 3, padding=1)
    self.conv3 = nn.Conv1d(256, 512, 3, padding=1)
    self.conv4 = nn.Conv1d(512, 1024, 3, padding=1)
    self.fc = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.embedding(x)
    x = x.permute(0, 2, 1)  # Switch dimensions for conv1d

    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = F.relu(self.conv4(x))

    x = F.max_pool1d(x, x.size(2))  # Global max pooling
    x = x.squeeze(2)

    x = self.fc(x)
    return x

##### **Train without Teacher**

For reference only, no need to run multiple times

In [16]:
import torch.optim as optim
from tqdm import tqdm

In [17]:
# Training function
def trainSimple(model, dataloader, criterion, optimizer, device):
  model.train()
  running_loss = 0.0
  correct_predictions = 0
  total_predictions = 0
  for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
    input_ids, labels = input_ids.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(input_ids)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, preds = torch.max(outputs, 1)
    correct_predictions += (preds == labels).sum().item()
    total_predictions += labels.size(0)

    running_loss += loss.item()

  loss = running_loss / len(dataloader)
  acc = correct_predictions / total_predictions
  return loss, acc

In [18]:
def evaluateSimple(model, dataloader, device):
  model.eval()
  correct_predictions = 0
  total_predictions = 0
  running_loss = 0.0
  with torch.no_grad():
    for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
      input_ids, labels = input_ids.to(device), labels.to(device)


      outputs = model(input_ids)
      loss = criterion(outputs, labels)

      _, preds = torch.max(outputs, 1)
      correct_predictions += (preds == labels).sum().item()
      total_predictions += labels.size(0)

      running_loss += loss.item()

    loss = running_loss / len(dataloader)
    acc = correct_predictions / total_predictions

  return loss, acc

In [ ]:
# Hyperparameters
vocab_size = len(tokenizer.vocab)
embed_dim = 128
n_classes = 4

# Create the model
model = SimpleCNN(vocab_size, embed_dim, n_classes)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
n_epochs = 20
best_acc = 0.0
for epoch in range(n_epochs):
    train_loss, train_acc = trainSimple(model, train_dataloader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{n_epochs}, train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    val_loss, val_acc = evaluateSimple(model, test_dataloader, device)
    print(f"test loss: {val_loss:.4f}, acc: {val_acc:.4f}")
    if best_acc < val_acc:
      best_acc = val_acc

100%|██████████| 938/938 [00:35<00:00, 26.38it/s]


Epoch 1/20, train loss: 0.5714, acc: 0.7721


100%|██████████| 60/60 [00:00<00:00, 76.81it/s]


test loss: 0.3609, acc: 0.8747


100%|██████████| 938/938 [00:35<00:00, 26.51it/s]


...

test loss: 0.6793, acc: 0.9041


100%|██████████| 938/938 [00:34<00:00, 26.84it/s]


Epoch 20/20, train loss: 0.0271, acc: 0.9913


100%|██████████| 60/60 [00:00<00:00, 75.82it/s]

test loss: 0.6836, acc: 0.8988


In [ ]:
print(f"best acc: {best_acc}")

best acc: 0.9086842105263158


#### Student 2

In [25]:
# even small cnn
class SimplierCNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, n_classes):
    super(SimplierCNN, self).__init__()

    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.conv1 = nn.Conv1d(embed_dim, 128, 3, padding=1)
    self.fc = nn.Linear(128, n_classes)

  def forward(self, x):
    x = self.embedding(x)
    x = x.permute(0, 2, 1)  # Switch dimensions for conv1d

    x = F.relu(self.conv1(x))

    x = F.max_pool1d(x, x.size(2))  # Global max pooling
    x = x.squeeze(2)

    x = self.fc(x)
    return x

##### **Train without Teacher**

For reference only, no need to run multiple times

In [26]:
# Hyperparameters
vocab_size = len(tokenizer.vocab)
embed_dim = 128
n_classes = 4

# Create the model
model = SimplierCNN(vocab_size, embed_dim, n_classes)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
# Training loop
n_epochs = 20
best_acc = 0.0
for epoch in range(n_epochs):
    train_loss, train_acc = trainSimple(model, train_dataloader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}/{n_epochs}, train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    val_loss, val_acc = evaluateSimple(model, test_dataloader, device)
    print(f"test loss: {val_loss:.4f}, acc: {val_acc:.4f}")
    if best_acc < val_acc:
      best_acc = val_acc

100%|██████████| 938/938 [00:04<00:00, 197.62it/s]


Epoch 1/20, train loss: 0.4973, acc: 0.8276


100%|██████████| 60/60 [00:00<00:00, 878.09it/s]


test loss: 0.3361, acc: 0.8866


100%|██████████| 938/938 [00:04<00:00, 218.86it/s]


...

test loss: 0.6329, acc: 0.8986


100%|██████████| 938/938 [00:04<00:00, 216.03it/s]


Epoch 20/20, train loss: 0.0118, acc: 0.9980


100%|██████████| 60/60 [00:00<00:00, 858.26it/s]

test loss: 0.6604, acc: 0.9007


In [28]:
print(f"best acc: {best_acc}")

best acc: 0.9092105263157895


## Training Teacher

In [ ]:
import torch.optim as optim
from tqdm import tqdm

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
  model.train()
  running_loss = 0.0
  running_acc = 0.0
  correct_predictions = 0
  total_predictions = 0
  for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    logits = model(input_ids, attention_mask, token_type_ids)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

    _, preds = torch.max(logits, 1)

    correct_predictions += (preds == labels).sum().item()
    total_predictions += labels.size(0)

    running_loss += loss.item()
  
  loss = running_loss / len(dataloader)
  acc = correct_predictions / total_predictions
  return loss, acc


In [ ]:
def evaluate(model, dataloader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    running_loss = 0.0
    with torch.no_grad():
      for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
          input_ids = input_ids.to(device)
          attention_mask = attention_mask.to(device)
          token_type_ids = token_type_ids.to(device)
          labels = labels.to(device)

          logits = model(input_ids, attention_mask, token_type_ids)
          loss = criterion(logits, labels)
          _, preds = torch.max(logits, 1)
          running_loss += loss.item()
          correct_predictions += (preds == labels).sum().item()
          total_predictions += labels.size(0)
      loss = running_loss / len(dataloader)
      acc = correct_predictions / total_predictions

    return loss, acc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_classes = 4
model_T = BERT_VDCNN(n_classes, freeze_bert=True, depth=9)
model_T.to(device)

optimizer = optim.Adam(model_T.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
num_epochs = 30
train_loss = []
train_acc = []
best_acc = 0.0
test_loss_record = []
test_acc_record = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    current_loss, current_acc = train(model_T, train_dataloader, optimizer, criterion, device)
    train_loss.append(current_loss)
    train_acc.append(current_acc)
    test_loss, test_accuracy = evaluate(model_T, test_dataloader, device)
    test_loss_record.append(test_loss)
    test_acc_record.append(test_accuracy)
    if best_acc < test_accuracy:
      best_acc = test_accuracy
      torch.save(model_T.state_dict(), 'best_teacher.pth')
    print(f'Train Loss: {current_loss}, Acc: {current_acc}')
    print(f"Test Loss: {test_loss},Test Accuracy: {test_accuracy:.4f}")
print(f'best test acc: {best_acc}')

Epoch 1/30


100%|██████████| 60/60 [00:27<00:00,  2.16it/s]


Train Loss: 1.4028130614204701, Acc: 0.7205166666666667
Test Loss: 0.34167931402722995,Test Accuracy: 0.8830
Epoch 2/30


100%|██████████| 60/60 [00:27<00:00,  2.17it/s]


Train Loss: 0.4320402960660361, Acc: 0.8678833333333333
Test Loss: 0.32749261235197386,Test Accuracy: 0.8861
Epoch 3/30


...

Train Loss: 0.1774194588475644, Acc: 0.9369166666666666
Test Loss: 0.21946321092545987,Test Accuracy: 0.9266
Epoch 29/30


100%|██████████| 60/60 [00:27<00:00,  2.17it/s]


Train Loss: 0.17480221632987197, Acc: 0.9379666666666666
Test Loss: 0.21774864867329596,Test Accuracy: 0.9326
Epoch 30/30


100%|██████████| 60/60 [00:27<00:00,  2.17it/s]

Train Loss: 0.17272642991547263, Acc: 0.9392916666666666
Test Loss: 0.22159153409302235,Test Accuracy: 0.9289
best test acc: 0.9326315789473684


### Utility Function for Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_train_curve(val_loss, val_acc, name_1, name_2):
  fig, ax1 = plt.subplots()

  color = 'tab:red'
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('Loss', color=color)
  ax1.plot(val_loss, color=color, linestyle='--', label=name_1)
  ax1.tick_params(axis='y', labelcolor=color)
  ax1.legend(loc='upper left')

  ax2 = ax1.twinx()
  color = 'tab:blue'
  ax2.set_ylabel('Accuracy', color=color)
  ax2.plot(val_acc, color=color, label=name_2)
  ax2.tick_params(axis='y', labelcolor=color)
  ax2.legend(loc='upper right')

  # Show the plot
  plt.show()

In [ ]:
#plot example
plot_train_curve(test_loss_record, test_acc_record, 'test loss', 'test acc')

## Train Student

Dist Loss

In [29]:
def intra_class_relation(y_s, y_t):
    return inter_class_relation(y_s.transpose(0, 1), y_t.transpose(0, 1))

def inter_class_relation(y_s, y_t):
    return 1 - pearson_correlation(y_s, y_t).mean()

def cosine_similarity(a, b, eps=1e-8):
    return (a * b).sum(1) / (a.norm(dim=1) * b.norm(dim=1) + eps)


def pearson_correlation(a, b, eps=1e-8):
    return cosine_similarity(a - a.mean(1).unsqueeze(1),
                             b - b.mean(1).unsqueeze(1), eps)



class DISTLoss(nn.Module):
    def __init__(self, beta=2.0, gamma=2.0, tau=1.0):
        super(DISTLoss, self).__init__()
        self.beta = beta
        self.gamma = gamma
        self.tau = tau

    def forward(self, z_s, z_t, labels):
        y_s = (z_s / self.tau).softmax(dim=1)
        y_t = (z_t / self.tau).softmax(dim=1)
       # print(y_s.shape)
       # print(y_t.shape)
        inter_loss = self.tau**2 * inter_class_relation(y_s, y_t)
        intra_loss = self.tau**2 * intra_class_relation(y_s, y_t)
        classification_loss = nn.CrossEntropyLoss()(z_s, labels)
        kd_loss = classification_loss+self.beta * inter_loss + self.gamma * intra_loss
        return kd_loss

In [30]:
# Training function
def trainWithT(model_S, model_T, dataloader, criterion, optimizer, device):
  model_S.train()
  model_T.eval()
  running_loss = 0.0
  correct_predictions = 0
  total_predictions = 0
  for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
    input_ids, labels = input_ids.to(device), labels.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    optimizer.zero_grad()
    studenet_logit = model_S(input_ids)
    teacher_logit = model_T(input_ids, attention_mask, token_type_ids)
    loss = criterion(studenet_logit, teacher_logit, labels)
    loss.backward()
    optimizer.step()

    _, preds = torch.max(studenet_logit, 1)
    correct_predictions += (preds == labels).sum().item()
    total_predictions += labels.size(0)

    running_loss += loss.item()

  loss = running_loss / len(dataloader)
  acc = correct_predictions / total_predictions
  return loss, acc

def evaluate(model, dataloader, device):
  model.eval()
  correct_predictions = 0
  total_predictions = 0
  running_loss = 0.0
  with torch.no_grad():
    for input_ids, attention_mask, token_type_ids, labels in tqdm(dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        logits = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(logits, labels)
        _, preds = torch.max(logits, 1)
        running_loss += loss.item()
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
    loss = running_loss / len(dataloader)
    acc = correct_predictions / total_predictions

  return loss, acc

##### Student 1

In [ ]:
# Hyperparameters
vocab_size = len(tokenizer.vocab)
embed_dim = 128
n_classes = 4

# Create the model
model_S = SimpleCNN(vocab_size, embed_dim, n_classes)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_S.to(device)

# Loss and optimizer
criterion_S = DISTLoss()
optimizer_S = optim.Adam(model_S.parameters(), lr=0.001)

Load teacher:

(reminder to check path）

In [ ]:
model_T = BERT_VDCNN(n_classes, freeze_bert=True, depth=9)
checkpoint_path = './best_teacher.pth' # Will need adjustment if model path or name is different
# Load the state_dict into model_T
model_T.load_state_dict(torch.load(checkpoint_path))
model_T.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_VDCNN(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

Train Loop, with teacher

In [ ]:
# Training loop
n_epochs = 20
best_acc = 0.0
for epoch in range(n_epochs):
    train_loss, train_acc = trainWithT(model_S, model_T, train_dataloader, criterion_S, optimizer_S, device)
    print(f"Epoch {epoch+1}/{n_epochs}, train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    val_loss, val_acc = evaluateSimple(model_S, test_dataloader, device)
    print(f"test loss: {val_loss:.4f}, acc: {val_acc:.4f}")
    if best_acc < val_acc:
      best_acc = val_acc

100%|██████████| 938/938 [09:29<00:00,  1.65it/s]


Epoch 1/20, train loss: 1.3036, acc: 0.7993


100%|██████████| 60/60 [00:00<00:00, 75.81it/s]


test loss: 0.3732, acc: 0.8746


100%|██████████| 938/938 [09:36<00:00,  1.63it/s]


...

test loss: 0.2983, acc: 0.9124


100%|██████████| 938/938 [09:34<00:00,  1.63it/s]


Epoch 20/20, train loss: 0.1578, acc: 0.9653


100%|██████████| 60/60 [00:00<00:00, 77.49it/s]

test loss: 0.2865, acc: 0.9161


##### Student 2

In [35]:
# Hyperparameters
vocab_size = len(tokenizer.vocab)
embed_dim = 128
n_classes = 4

# Create the model
model_S = SimplierCNN(vocab_size, embed_dim, n_classes)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_S.to(device)

# Loss and optimizer
criterion_S = DISTLoss()
optimizer_S = optim.Adam(model_S.parameters(), lr=0.001)

Load teacher:

(reminder to check path）

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
model_T = BERT_VDCNN(n_classes, freeze_bert=True, depth=9)
checkpoint_path = './drive/MyDrive/DeepLearn-Project/NLP_Attempts/best_teacher.pth' # Will need adjustment if model path or name is different
# Load the state_dict into model_T
model_T.load_state_dict(torch.load(checkpoint_path))
model_T.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_VDCNN(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [36]:
# Training loop
n_epochs = 20
best_acc = 0.0
for epoch in range(n_epochs):
    train_loss, train_acc = trainWithT(model_S, model_T, train_dataloader, criterion_S, optimizer_S, device)
    print(f"Epoch {epoch+1}/{n_epochs}, train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    val_loss, val_acc = evaluateSimple(model_S, test_dataloader, device)
    print(f"test loss: {val_loss:.4f}, acc: {val_acc:.4f}")
    if best_acc < val_acc:
      best_acc = val_acc

100%|██████████| 938/938 [08:44<00:00,  1.79it/s]


Epoch 1/20, train loss: 1.3345, acc: 0.8265


100%|██████████| 60/60 [00:00<00:00, 607.85it/s]


test loss: 0.3723, acc: 0.8847


100%|██████████| 938/938 [08:49<00:00,  1.77it/s]


...

test loss: 0.3154, acc: 0.9083


100%|██████████| 938/938 [08:49<00:00,  1.77it/s]


Epoch 20/20, train loss: 0.1505, acc: 0.9682


100%|██████████| 60/60 [00:00<00:00, 804.30it/s]

test loss: 0.2957, acc: 0.9146


In [37]:
print(f'best acc: {best_acc}')

best acc: 0.9146052631578947
